In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pdb
import math
import torch.nn.init as init
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from masked_cross_entropy import *
from attn import Attn
import pdb
import random

from enc_dec import EncoderRNN, DecoderRNN

In [2]:
z_size = 10
epsilon = torch.zeros(z_size).cuda()
epsilon.normal_(0, 1) # 0 mean unit variance gaussian




 0.4067
-0.1418
-0.2261
-0.0026
 1.0591
 0.1637
-1.8403
 0.1628
-0.8616
-0.2907
[torch.cuda.FloatTensor of size 10 (GPU 0)]

In [3]:
log_sigma = Variable(torch.randn(z_size)).cuda()
mu = Variable(torch.randn(z_size)).cuda()

Variable(epsilon * torch.exp(log_sigma.data*0.5)+mu.data)

Variable containing:
 1.4711
 0.0977
 1.5382
 1.4021
 1.7067
 0.4583
-3.0813
 0.7612
 0.1530
-0.8584
[torch.cuda.FloatTensor of size 10 (GPU 0)]

In [4]:
n_layers = 3
bs = 32
enc_hidden_dim = 50
vocab_size = 20
enc_h = Variable(torch.randn(n_layers, bs, enc_hidden_dim))
print(enc_h.size())
linear_mu = nn.Linear(enc_hidden_dim, z_size)
linear_sigma = nn.Linear(enc_hidden_dim, z_size)

# use the last hidden layer's output
mu = linear_mu(enc_h[-1].squeeze())
log_sigma = linear_sigma(enc_h[-1].squeeze())
print(mu.size())
print(log_sigma.size())

epsilon = torch.zeros(z_size)
epsilon.normal_(0, 1) # 0 mean unit variance gaussian
z = Variable(epsilon*torch.exp(log_sigma.data*0.5)+mu.data)
print(z.size()) # => batch_size x z_size

torch.Size([3, 32, 50])
torch.Size([32, 10])
torch.Size([32, 10])
torch.Size([32, 10])


In [17]:
max_seq_len = 35
all_decoder_outputs = Variable(torch.randn(max_seq_len, bs, vocab_size))
# ref: http://pytorch.org/docs/master/_modules/torch/nn/modules/loss.html
# Target: :math:`(N)` where each value is `0 <= targets[i] <= C-1`
target = Variable(torch.randn(max_seq_len, bs)).long()
target.data[:] = 0

x = all_decoder_outputs

In [30]:
from torch.nn import functional as F


print(all_decoder_outputs.size())
print(target.size())
loss = 0
loss_fn = nn.CrossEntropyLoss()

for t in range(max_seq_len):
    #print(x[t].size())
    #print(target[t].size())
    #print(target[t].view(target[t].size()[0], 1).size())
    #loss += loss_fn(x[t], target[t].view(target[t].size()[0], 1))
    loss += loss_fn(x[t], target[t])
    #print(type(loss))
    #print(loss)
    
#CE = nn.CrossEntropyLoss(x, target)
#CE = F.cross_entropy(x, target)
print('='*50)
print(loss)
print(loss/float(max_seq_len))


#KLD = -0.5 * torch.sum(1 + log_sigma - mu*mu - torch.exp(log_sigma))
#KLD /= bs # normalize
#print(KLD)
tmp = 1 + log_sigma - mu*mu - torch.exp(log_sigma)
print(tmp.size()) # => bs x z_size
KLD = (-0.5 * torch.sum(1 + log_sigma - mu*mu - torch.exp(log_sigma), 1)).mean()
print('='*50)
print(KLD)


print( (loss + KLD).size() )

torch.Size([35, 32, 20])
torch.Size([35, 32])
Variable containing:
 120.8209
[torch.FloatTensor of size 1]

Variable containing:
 3.4520
[torch.FloatTensor of size 1]

torch.Size([32, 10])
Variable containing:
 2.8358
[torch.FloatTensor of size 1]

torch.Size([1])
